### Secure tool calling with Arcade.dev

This notebook is going to close one of the biggest gaps between demo and production agents: Secure tool-calling

When your agents work well in your computer, they are excellent personal assistants, but scaling that up to many users is not easy, as the security assumptions from a local deployment do not apply to agents at scale. Personal Access Tokens simply won't cut it for multiple users. Even if you encapsulate all of the functionality in a remote MCP server, tool-level auth will require you to implement the auth flow for all the providers that your agent relies on.

Arcade solves this by providing a unified platform for agentic tool calling and execution. It will handle the auth flow for you offering a secure multi-user solution for your agents.

In this notebook we will learn how to use Arcade and LangGraph to :-

- Build agents
- Give tools that can interact with
    - GMail
    - Slack
    - Notion
- Implement safety guardrails when calling specific tools (Human-in-the-Loop)

## Development Environment Setup

Before implementing our multi-user agent system, we need to establish a proper development environment with the necessary dependencies. The following installation includes LangGraph for agent orchestration, LangChain-Arcade for tool integration, and the core LangChain library with OpenAI support.

In [19]:
# !pip3 install langgraph langchain-arcade langchain langchain-openai
# Install required packages
!pip3 install langchain langchain-openai langgraph langchain-arcade python-dotenv

Python(40696) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


INFO: pip is looking at multiple versions of langchain-arcade to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_arcade-1.3.1-py3-none-any.whl.metadata (6.6 kB)
  Using cached arcadepy-1.3.1-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_arcade-1.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached langchain_arcade-1.2.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached arcadepy-1.1.1-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_arcade-1.1.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached langchain_arcade-1.0.0-py3-none-any.whl.metadata (2.0 kB)
  Using cached langchain_arcade-0.1.2-py3-none-any.whl.metadata (2.1 kB)
  Using cached arcadepy-0.1.2-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_arcade-0.1.1-py3-none-any.whl.metadata (2.1 kB)
INFO: pip is still looking at multiple versions of langchain-arcade to determine which version is compatible with other requirements. This could

## API Key Configuration

Our tutorial requires two essential API keys for operation. You will need an [OpenAI API](https://platform.openai.com/signup) key, as well as an [Arcade API](https://api.arcade.dev/signup?utm_source=github&utm_medium=notebook&utm_campaign=nir_diamant&utm_content=tutorial) key for this tutorial. Both services offer straightforward registration processes, with Arcade specifically designed to simplify the integration of external tools into AI applications.

In [20]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file (create this with your API key)
load_dotenv()

# Set OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# Set Arcade API key
os.environ["ARCADE_API_KEY"] = os.getenv('ARCADE_API_KEY')

## User Identity Configuration

The Arcade platform requires user identification to properly manage tool authorizations and maintain security boundaries between different users. This identifier must correspond to the email address used during Arcade account creation, ensuring that tool permissions and OAuth tokens are correctly associated with the appropriate user account.

In [21]:
# Set ARCADE user id which is the email address used furing Arcade account creation
os.environ["ARCADE_USER_ID"] = os.getenv('ARCADE_USER_ID')


# Simple Conversational Agent

We begin our journey by implementing a basic conversational agent that demonstrates core LangGraph functionality without external tool dependencies. This foundational agent provides conversational capabilities with short-term memory, allowing it to maintain context throughout a conversation while establishing the architectural patterns we'll extend throughout this tutorial.

## Core Agent Implementation

The following implementation creates a ReAct-style agent using [LangGraph and Arcade](https://docs.arcade.dev/home/langchain/use-arcade-tools#create-a-react-style-agent?utm_source=github&utm_medium=notebook&utm_campaign=nir_diamant&utm_content=tutorial). We configure it with conversation memory through a MemorySaver checkpointer, enabling the agent to remember previous interactions within the same conversation thread. The agent receives a clear prompt defining its helpful and concise personality, along with instructions for handling unclear requests.

In [22]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage
import uuid

# create a checkpointer to persist the graph's state
checkpointer = MemorySaver()

agent_a = create_agent(
    model="openai:gpt-5-mini",
    system_prompt="You are a helpful assistant that can help with everyday tasks."
           " If the user's request is confusing you must ask them to clarify"
           " their intent, and fulfill the instruction to the best of your"
           " ability. Be concise and friendly at all times.",
    tools=[], # no tools for now!
    checkpointer=checkpointer
)

## Agent Interaction Utility

To facilitate consistent interaction with our agents throughout this tutorial, we implement a utility function that streams agent responses and displays them in a readable format. This function processes the graph's streaming output and presents the latest message from each interaction cycle, providing immediate feedback during agent conversations.

In [53]:
from langgraph.graph.state import CompiledStateGraph
def run_graph(graph: CompiledStateGraph, config, input):
    
    for event in graph.stream(input, config=config, stream_mode="values"):
        if "messages" in event:
            event["messages"][-1].pretty_print()

## Interactive Chat Interface

The following implementation provides a complete interactive chat interface for testing our basic agent. The system generates a unique conversation thread identifier for each session, enabling memory persistence across multiple exchanges within the same conversation. Users can engage naturally with the agent and terminate the session by typing "exit".

In [24]:
# the configuration helps LangGraph keep track of conversations and interrups
# While it's not needed for this agent. The agent will remember different
# conversations based on the thread_id. This code generates a random id every
# time you run the cell, but you can hardcode the thread_id if you want to
# test the memory.
config = {
    "configurable": {
        "thread_id": uuid.uuid4()
    }
}
while True:
    user_input = input("👤: ")
    # let's use "exit" as a safe way to break the infinite loop
    if user_input.lower() == "exit":
        break

    user_message = {"messages": [HumanMessage(content=user_input)]}
    run_graph(agent_a, config, user_message)



================================ Human Message =================================

Hi, can you let me know about any open-source tools or libraries that can streamline the proces of auth flow for connect different APIs with LLMs?
================================== Ai Message ==================================

Great — here are open‑source tools and libraries, organized by role, that will speed up building and managing auth flows when connecting external APIs to LLMs (or any backend/agent):

1) OAuth/OIDC client libraries (use to implement auth flows, refresh tokens, etc.)
- Python: Authlib, requests-oauthlib — mature, support OAuth2/OIDC flows and token refresh.
- Node.js: openid-client, simple-oauth2, passport.js (strategy ecosystem) — easy to wire into web apps.
- Go: golang.org/x/oauth2 — standard, lightweight.
- Java: Spring Security OAuth/OIDC — full-featured for Spring apps.

2) Identity / Authorization servers (self-hosted providers for issuing tokens and managing users)
- Keyclo

## Testing Agent Limitations / Demonstrating Authentication Requirements

The following test illustrates the agent's complete inability to access private, authenticated data sources. When asked to summarize personal emails, the agent cannot proceed without proper authentication mechanisms and authorized access to external services. This limitation highlights the critical need for secure tool integration in production agent systems.

To understand the boundaries of our basic agent, we'll test it with requests that require external data access. The following test demonstrates the agent's inability to provide current date information, as most language models lack real-time data access and may provide outdated or inaccurate temporal information.

In [27]:
config = {
    "configurable": {
        "thread_id": uuid.uuid4()
    }
}
print(f'thread_id = {config["configurable"]["thread_id"]}')

prompt = "summarize my latest 3 emails please"
user_message = {"messages": [HumanMessage(content=prompt)]}
run_graph(agent_a, config, user_message)

thread_id = 055a7ebe-17ed-4e23-b9eb-fa2c5477658e
================================ Human Message =================================

summarize my latest 3 emails please
================================== Ai Message ==================================

I don’t have access to your inbox. If you’d like me to summarize your latest three emails, please paste the emails (or the subject/sender/date and body for each) here — or forward the text of each message.

How to paste (quick template you can copy for each email):
- Subject:
- From:
- Date:
- Body:

Tell me which summary format you want:
- Very short: one line per email (main point).
- Short: 3–4 bullets per email (key points + one action).
- Detailed: summary, action items, deadlines, suggested replies.

Also tell me if you want:
- Highlighted action items and deadlines
- Suggested reply drafts for any messages
- Priority ranking

Paste the emails and your preferred format, and I’ll summarize them.


# Tool Integration with Secure Authentication

Having established our basic agent architecture, we now address the core challenge of enabling secure access to external services. This section demonstrates how Arcade.dev solves the complex problem of tool-level authentication, providing a streamlined approach to OAuth integration that scales across multiple users and services.

## Arcade Client Initialization

We begin by establishing connections to the Arcade platform through both the core client and the LangChain integration layer. The ToolManager serves as our primary interface for configuring and authorizing tools, while the Arcade client handles the underlying authentication infrastructure.

In [28]:
from langchain_arcade import ToolManager
from arcadepy import Arcade

arcade_client = Arcade(api_key=os.getenv("ARCADE_API_KEY"))
manager = ToolManager(client=arcade_client)

## Gmail Tool Configuration

Our first tool integration focuses on Gmail access, specifically the email listing capability that our basic agent could not provide. The Gmail_ListEmails tool enables our agent to retrieve and analyze email data, but requires proper user authorization before it can access private email accounts.

In [47]:
gmail_tool = manager.init_tools(tools=["Gmail_ListEmails"])[0]

## Authorization Utility Function

To streamline the authorization process throughout this tutorial, we implement a reusable function that handles OAuth flow initiation and completion. For reading our email, however, we need to give our app permissions to read it in a secure way. Arcade lets us do this easily by [handling the OAuth2 for us](https://docs.arcade.dev/home/auth/how-arcade-helps?utm_source=github&utm_medium=notebook&utm_campaign=nir_diamant&utm_content=tutorial). This function checks the current authorization status for a specific tool and user combination, initiating the OAuth process when necessary and waiting for user completion of the authorization flow.

In [48]:
def authorize_tool(tool_name, user_id, manager):
    # This line will check if this user is authorized to use the
    # tool, and return a response that we can use if the user
    # did not authorize the tool yet.
    auth_response = manager.authorize(
        tool_name=tool_name,
        user_id=user_id
    )
    if auth_response.status != "completed":
        print(f"The app wants to use the {tool_name} tool.\n"
              f"Please click this url to authorize it {auth_response.url}")
        # wait until the user authorizes
        manager.wait_for_auth(auth_response.id)


## Gmail Authorization Process

The following cell initiates the authorization process for Gmail access. If the user has not previously granted permissions, Arcade will provide an OAuth URL for completing the authorization. Once authorized, the permission persists for future sessions, eliminating the need for repeated authorization flows.

In [67]:
authorize_tool(gmail_tool.name, os.getenv("ARCADE_USER_ID"), manager)

## Enhanced Agent with Gmail Capabilities

With Gmail authorization complete, we can now create an enhanced agent that incorporates email access capabilities. This agent retains all the conversational abilities of our basic implementation while adding the power to interact with authenticated email services. Notice the updated prompt that explicitly mentions Gmail capabilities and the inclusion of the user_id in the configuration for tool execution.

In [64]:
# define a new agent, this time with access to our tool!
agent_b = create_agent(
    model="openai:gpt-5",
    system_prompt="You are a helpful assistant that can help with everyday tasks."
           " If the user's request is confusing you must ask them to clarify"
           " their intent, and fulfill the instruction to the best of your"
           " ability. Be concise and friendly at all times."
           # It's useful to let the agent know about the tools it has at its disposal.
           " Use the Gmail tools that you have to address requests about emails.",
    tools=[gmail_tool], # we pass the tool we previously authorized.
    checkpointer=checkpointer
)

config = {
    "configurable": {
        "thread_id": uuid.uuid4(),
        "user_id": os.getenv("ARCADE_USER_ID") # When using Arcade tools, we must provide the user_id on the LangGraph config, so Arcade can execute the tool invoked by the agent.
    }
}
print(f'thread_id = {config["configurable"]["thread_id"]}')
print(f'user_id = {config["configurable"]["user_id"]}')

# we're using the same prompt we use before, but we're swapping the agent
prompt = "summarize my latest 3 emails please"
user_message = {"messages": [HumanMessage(content=prompt)]}
run_graph(agent_b, config, user_message)


thread_id = d062f6ce-0563-421e-a8b6-9de3a33ccba8
user_id = abdullahmakhdoom1998@gmail.com
================================ Human Message =================================

summarize my latest 3 emails please
================================== Ai Message ==================================
Tool Calls:
  Gmail_ListEmails (call_za8N36pUOekT3W9fDDdLS0rf)
 Call ID: call_za8N36pUOekT3W9fDDdLS0rf
  Args:
    n_emails: 3
================================= Tool Message =================================
Name: Gmail_ListEmails

{"emails": [{"body": "[image: Google] You allowed Arcade.dev access to some of your Google Account data abdullahmakhdoom1998@gmail.com If you didn’t allow Arcade.dev access to some of your Google Account data, someone else may be trying to access your Google Account data. Take a moment now to check your account activity and secure your account. Check activity To make changes at any time to the access that Arcade.dev has to your data, go to your Google Account You can also se